## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-14-56-50 +0000,bbc,European allies back Denmark over Trump's thre...,https://www.bbc.com/news/articles/c07xkeee2k3o...
1,2026-01-06-14-53-02 +0000,nypost,Female students accuse transgender athlete at ...,https://nypost.com/2026/01/06/us-news/students...
2,2026-01-06-14-51-00 +0000,wsj,Lockheed Martin to More Than Triple PAC-3 Miss...,https://www.wsj.com/politics/national-security...
3,2026-01-06-14-48-32 +0000,nyt,"Bela Tarr, Titan of Slow-Moving Cinema, Dies a...",https://www.nytimes.com/2026/01/06/movies/bela...
4,2026-01-06-14-47-45 +0000,nyt,Will Welch to Leave GQ After 7 Years as Its To...,https://www.nytimes.com/2026/01/06/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,venezuela,58
3,trump,50
273,maduro,48
143,new,20
60,oil,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
244,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,185
274,2026-01-05-19-32-24 +0000,nyt,"Who Is Delcy Rodríguez, Venezuela’s New Interi...",https://www.nytimes.com/2026/01/05/world/ameri...,165
279,2026-01-05-19-05-33 +0000,nyt,Trump’s Goals for Reviving Venezuela’s Oil Ind...,https://www.nytimes.com/2026/01/05/business/en...,150
265,2026-01-05-20-11-39 +0000,nypost,Elon Musk’s Starlink offers free internet serv...,https://nypost.com/2026/01/05/business/elon-mu...,147
55,2026-01-06-12-40-51 +0000,bbc,Trump claims US oil firms could be 'up and run...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,147


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
244,185,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
91,88,2026-01-06-10-33-43 +0000,nypost,"Tim Walz’s firebrand daughter, Hope, admits th...",https://nypost.com/2026/01/06/us-news/tim-walz...
167,64,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
75,54,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
197,44,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
239,43,2026-01-05-21-36-00 +0000,wsj,U.S. companies will benefit under a new global...,https://www.wsj.com/politics/policy/oecd-corpo...
110,34,2026-01-06-09-40-00 +0000,wsj,Saudi Arabia Cuts Flagship Oil Price to Asia f...,https://www.wsj.com/business/energy-oil/saudi-...
177,33,2026-01-06-01-37-29 +0000,nypost,Newsom claims he was ‘physically’ in middle of...,https://nypost.com/2026/01/05/us-news/newsom-b...
93,32,2026-01-06-10-30-23 +0000,bbc,Nvidia unveils self-driving car tech as it see...,https://www.bbc.com/news/articles/c0jv1vd571wo...
45,30,2026-01-06-13-09-00 +0000,wsj,Next PLC Lifts Outlook as International Sales ...,https://www.wsj.com/business/retail/next-plc-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
